Try to make a model that is both good at size and precision. This time we will try many possibilities.

In [19]:
import pandas as pd
import numpy as np

In [20]:
final_gird_dataset = pd.read_csv('final_grid_dataset_final.csv')
final_gird_dataset.head(5)

,grid_code,time_stamp,taxi_density,pm2.5_aqi,humidity,wind_direction,temp,wind_speed,wind_gust,pressure,weather_id
0,0@7,1680310800,0.0,52.226051,62,S,287.594444,4.4704,0.0,1009.482859,804
1,0@8,1680310800,0.0,52.226051,62,S,287.594444,4.4704,0.0,1009.482859,804
2,0@9,1680310800,0.0,68.886052,62,S,287.594444,4.4704,0.0,1009.482859,804
3,1@9,1680310800,0.0,68.886052,62,S,287.594444,4.4704,0.0,1009.482859,804
4,3@6,1680310800,0.0,52.226051,62,S,287.594444,4.4704,0.0,1009.482859,804


In [21]:
from sklearn.base import BaseEstimator, TransformerMixin

In [22]:
class WindDirectionOneHotEncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.directional_strings = ['S', 'SSW', 'SW', 'SSE', 'WNW', 'NNW', 'WSW', 'NW', 'N', 'NE', 'ENE', 'E', 'ESE', 'SE', 'NNE', 'W']
    
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
            
        X = X.copy()
        X['wind_direction'] = X['wind_direction'].apply(lambda x: x if x in self.directional_strings else 'other')

        X_encoded = pd.get_dummies(X, columns=['wind_direction'], prefix='', prefix_sep='')

        if 'other' in X_encoded.columns:
            X_encoded = X_encoded.drop(columns=['other'])

        for col in self.directional_strings:
            if col not in X_encoded.columns:
                X_encoded[col] = 0

        X_encoded = X_encoded[self.directional_strings]
        X_encoded = X_encoded.astype(int)

        return X_encoded

In [23]:
class WindDirectionTwoDimensionalEncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.directional_map = {
            'N': (1.0, 0.0),        # 0°
            'NNE': (0.966, 0.259),  # 22.5°
            'NE': (0.866, 0.5),     # 45°
            'ENE': (0.707, 0.707),  # 67.5°
            'E': (0.5, 0.866),      # 90°
            'ESE': (0.259, 0.966),  # 112.5°
            'SE': (0.0, 1.0),       # 135°
            'SSE': (-0.259, 0.966), # 157.5°
            'S': (-0.5, 0.866),     # 180°
            'SSW': (-0.707, 0.707), # 202.5°
            'SW': (-0.866, 0.5),    # 225°
            'WSW': (-0.966, 0.259), # 247.5°
            'W': (-1.0, 0.0),       # 270°
            'WNW': (-0.966, -0.259),# 292.5°
            'NW': (-0.866, -0.5),   # 315°
            'NNW': (-0.707, -0.707) # 337.5°
        }
    
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)

        X = X.copy()

        X['cos'] = X['wind_direction'].apply(lambda x: self.directional_map.get(x, (0.0, 0.0))[0])
        X['sin'] = X['wind_direction'].apply(lambda x: self.directional_map.get(x, (0.0, 0.0))[1])

        return X[['cos', 'sin']]

In [24]:
class WeatherIdEncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.weather_ids = [804, 500, 741, 803, 801, 800, 200, 501, 721, 300, 211, 502, 711, 212, 701, 600, 616, 612, 511, 601, 602, 301]

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
            
        X = X.copy()
        X['weather_id'] = X['weather_id'].apply(lambda x: x if x in self.weather_ids else 'other')

        X_encoded = pd.get_dummies(X, columns=['weather_id'], prefix='', prefix_sep='')

        if 'other' in X_encoded.columns:
            X_encoded = X_encoded.drop(columns=['other'])

        for weather_id in self.weather_ids:
            if str(weather_id) not in X_encoded.columns:
                X_encoded[str(weather_id)] = 0

        X_encoded = X_encoded[[str(weather_id) for weather_id in self.weather_ids]]
        X_encoded = X_encoded.astype(int)

        return X_encoded

In [25]:
class GridCodeTwoDimensionalEncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        
        X = X.copy()
        X[['grid_x', 'grid_y']] = X['grid_code'].str.split('@', expand=True).astype(int)
        return X.drop(columns=['grid_code'])

In [26]:
class TimestampEncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)

        X = X.copy()

        X['month'] = pd.to_datetime(X['time_stamp'], unit='s').dt.month
        X['day_of_week'] = pd.to_datetime(X['time_stamp'], unit='s').dt.weekday
        X['hour_of_day'] = pd.to_datetime(X['time_stamp'], unit='s').dt.hour

        X = X.drop(columns=['time_stamp'])
        return X

Split

In [27]:
from sklearn.model_selection import train_test_split

In [28]:
count = 0
for grid_code, group in final_gird_dataset.groupby('grid_code'):
    train_set_one_grid, test_set_one_grid = train_test_split(group, test_size=0.2, random_state=42)

    if count == 0:
        train_set = train_set_one_grid
        test_set = test_set_one_grid
    else:
        train_set = pd.concat([train_set, train_set_one_grid], axis=0)
        test_set = pd.concat([test_set, test_set_one_grid], axis=0)

    count += 1

print(f'Total: {count} grid_codes.')
print(f'test set: {len(test_set)}')
print(f'train set: {len(train_set)}')

Total: 358 grid_codes.
test set: 608600
train set: 2434042


Defining inputs and outputs

In [30]:
taxi_model_inputs = ['grid_code', 'time_stamp', 'humidity', 'wind_direction', 'temp', 'wind_speed', 'wind_gust', 'pressure', 'weather_id']
taxi_model_output = 'taxi_density' 
aqi_model_inputs = ['grid_code', 'time_stamp', 'taxi_density', 'humidity', 'wind_direction', 'temp', 'wind_speed', 'wind_gust', 'pressure', 'weather_id']
aqi_model_output = 'pm2.5_aqi'

Batch experiments
Encoder configurations

In [31]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
import itertools
from copy import deepcopy

In [32]:
wind_direction_encoders = [WindDirectionOneHotEncoder(), WindDirectionTwoDimensionalEncoder()]
timestamp_encoders = [None, TimestampEncoder()]
weather_id_encoder = WeatherIdEncoder()
grid_code_encoder = GridCodeTwoDimensionalEncoder()

Model configuration

In [33]:
models = {
    'linear_regression': "LinearRegression()",
    'decision_tree': "DecisionTreeRegressor()",
    'random_forest_small': "RandomForestRegressor(n_estimators=50, max_depth=10, min_samples_split=20, min_samples_leaf=10, n_jobs=-1, random_state=42)",
    'random_forest_complex': "RandomForestRegressor(n_estimators=100, max_depth=70, min_samples_split=5, min_samples_leaf=2, n_jobs=-1, random_state=42)"
}

In [34]:
class PrintData(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):          
        print("Transformed Data:")
        print(X[1])
        return X

In [35]:
def run_experiments(train_set, test_set, wind_dir_encoders, timestamp_encoders, models):
    results = []
    
    for model_name, model in models.items():
        
        print(f'------ {model_name} ------')
        count = 0
        
        for wind_dir_encoder, timestamp_encoder in itertools.product(wind_dir_encoders, timestamp_encoders):
            count += 1
            print(f'configuration {count}....')
            
            train_set = train_set.copy()
            test_set = test_set.copy()
            
            preprocessors = [('wind_direction', wind_dir_encoder, ['wind_direction']), 
                                  ('weather_id', weather_id_encoder, ['weather_id']), 
                                  ('grid_code', grid_code_encoder, ['grid_code'])]
            
            if timestamp_encoder:
                preprocessors.append(('time_stamp', timestamp_encoder, ['time_stamp']))
                
            # Taxi model
            taxi_pipeline = Pipeline([
                ('preprocessor', ColumnTransformer(deepcopy(preprocessors), remainder='passthrough')),
                ('scaler', StandardScaler()),
                ('model', eval(model))
            ])
            
            taxi_pipeline.fit(train_set[taxi_model_inputs], train_set[taxi_model_output])
            
            # AQI model
            aqi_pipeline = Pipeline([
                ('preprocessor', ColumnTransformer(deepcopy(preprocessors), remainder='passthrough')),
                ('scaler', StandardScaler()),
                ('model', eval(model))
            ])
            
            aqi_pipeline.fit(train_set[aqi_model_inputs], train_set[aqi_model_output])
            
            # Test 1
            aqi_test_inputs = test_set[aqi_model_inputs].copy()
            aqi_test_inputs['taxi_density'] = taxi_pipeline.predict(test_set[taxi_model_inputs])
            cascade_preds = aqi_pipeline.predict(aqi_test_inputs)
            cascade_rmse = root_mean_squared_error(test_set[aqi_model_output], cascade_preds)
            
            del taxi_pipeline, aqi_pipeline
            
            # Non cascade model            
            non_cascade_pipeline = Pipeline([
                ('preprocessor', ColumnTransformer(deepcopy(preprocessors), remainder='passthrough')),
                ('scaler', StandardScaler()),
                ('model', eval(model))
            ])
            
            non_cascade_pipeline.fit(train_set[taxi_model_inputs], train_set[aqi_model_output])
            
            # Test 2
            non_cascade_preds = non_cascade_pipeline.predict(test_set[taxi_model_inputs])
            non_cascade_rmse = root_mean_squared_error(test_set[aqi_model_output], non_cascade_preds)
            
            del non_cascade_pipeline
            
            results.append({
                'model': model_name,
                'wind_direction_encoder': wind_dir_encoder.__class__.__name__,
                'timestamp_encoder': timestamp_encoder.__class__.__name__ if timestamp_encoder else 'None',
                'cascade_rmse': cascade_rmse,
                'non_cascade_rmse': non_cascade_rmse
            })
    
    return results

Execute the test.

In [36]:
experiment_results = run_experiments(train_set, test_set, wind_direction_encoders, timestamp_encoders, models)

results_df = pd.DataFrame(experiment_results)
results_df.to_csv('model_experiment_results.csv', index=False)

------ linear_regression ------
configuration 1....
configuration 2....
configuration 3....
configuration 4....
------ decision_tree ------
configuration 1....
configuration 2....
configuration 3....
configuration 4....
------ random_forest_small ------
configuration 1....
configuration 2....
configuration 3....
configuration 4....
------ random_forest_complex ------
configuration 1....
configuration 2....
configuration 3....
configuration 4....


The best choice is a random forest model with complex parameters, no timestamp encoder and non-cascade structure.

In [37]:
import joblib

In [38]:
def train_model_on_full_set(dataset, wind_dir_encoder, timestamp_encoder, model, is_cascade):
    preprocessors = [('wind_direction', wind_dir_encoder, ['wind_direction']), 
                          ('weather_id', weather_id_encoder, ['weather_id']), 
                          ('grid_code', grid_code_encoder, ['grid_code'])]
    
    if timestamp_encoder:
        preprocessors.append(('time_stamp', timestamp_encoder, ['time_stamp']))
    
    if is_cascade:
        taxi_pipeline = Pipeline([
            ('preprocessor', ColumnTransformer(deepcopy(preprocessors), remainder='passthrough')),
            ('scaler', StandardScaler()),
            ('model', eval(model))
        ])
        taxi_pipeline.fit(dataset[taxi_model_inputs], dataset[taxi_model_output])
        
        joblib.dump(taxi_pipeline, 'taxi_density_model.pkl')
        
        aqi_pipeline = Pipeline([
            ('preprocessor', ColumnTransformer(deepcopy(preprocessors), remainder='passthrough')),
            ('scaler', StandardScaler()),
            ('model', eval(model))
        ])
        aqi_pipeline.fit(dataset[aqi_model_inputs], dataset[aqi_model_output])
        
        joblib.dump(aqi_pipeline, 'aqi_model.pkl')
    
    else:
        non_cascade_pipeline = Pipeline([
            ('preprocessor', ColumnTransformer(deepcopy(preprocessors), remainder='passthrough')),
            ('scaler', StandardScaler()),
            ('model', eval(model))
        ])
        non_cascade_pipeline.fit(dataset[taxi_model_inputs], dataset[aqi_model_output])
        
        joblib.dump(non_cascade_pipeline, 'non_cascade_model.pkl')

In [39]:
train_model_on_full_set(dataset=final_gird_dataset, 
                        wind_dir_encoder=WindDirectionTwoDimensionalEncoder(),
                        timestamp_encoder=None,
                        model='RandomForestRegressor(n_estimators=100, max_depth=70, min_samples_split=5, min_samples_leaf=2, n_jobs=-1, random_state=42)',
                        is_cascade=False)